In [1]:
import tensorflow as tf
import sonnet as snt

from PIL import Image, ImageOps

import numpy as np

import os

import i3d

W0725 16:05:37.826516 139983842342528 deprecation_wrapper.py:119] From /usr/lib/python3.7/site-packages/sonnet/python/custom_getters/restore_initializer.py:27: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0725 16:05:38.805845 139983842342528 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
image_size = 224
num_class = 20

sample_path = {
    'rgb': 'data/v_CricketShot_g04_c01_rgb.npy',
    'flow': 'data/v_CricketShot_g04_c01_flow.npy',
}

checkpoints = {
    'rgb': 'data/checkpoints/rgb_scratch/model.ckpt',
    'flow': 'data/checkpoints/flow_scratch/model.ckpt',
    'rgb_imagenet': 'data/checkpoints/rgb_imagenet/model.ckpt',
    'flow_imagenet': 'data/checkpoints/flow_imagenet/model.ckpt',
}

def extract_feat(rgb_batch, flow_batch):
    
    rgb_input = tf.placeholder(tf.float32, shape=(1,None,image_size,image_size,3))
    flow_input = tf.placeholder(tf.float32, shape=(1,None,image_size,image_size,2))
    with tf.variable_scope('RGB'):
        rgb_model = i3d.InceptionI3d(num_class+1, spatial_squeeze=True, final_endpoint='Mixed_5c')
        rgb_mixed5c, _ = rgb_model(rgb_input, is_training=False, dropout_keep_prob=1.0)
        rgb_feat = tf.nn.avg_pool3d(rgb_mixed5c, ksize=[1, 2, 7, 7, 1],
                             strides=[1, 1, 1, 1, 1], padding=snt.VALID)

    rgb_variable_map = {}
    for variable in tf.global_variables():
        if variable.name.split('/')[0] == 'RGB':
            rgb_variable_map[variable.name.replace(':0', '')] = variable
    rgb_saver = tf.train.Saver(var_list=rgb_variable_map, reshape=True)
    
    with tf.variable_scope('Flow'):
        flow_model = i3d.InceptionI3d(num_class+1,spatial_squeeze=True, final_endpoint='Mixed_5c')
        flow_mixed5c, _ = flow_model(flow_input, is_training=False, dropout_keep_prob=1.0)
        flow_feat = tf.nn.avg_pool3d(flow_mixed5c, ksize=[1, 2, 7, 7, 1],
                             strides=[1, 1, 1, 1, 1], padding=snt.VALID)
        
    flow_variable_map = {}
    for variable in tf.global_variables():
        if variable.name.split('/')[0] == 'Flow':
            flow_variable_map[variable.name.replace(':0', '')] = variable
    flow_saver = tf.train.Saver(var_list=flow_variable_map, reshape=True)
    
    with tf.Session() as sess:
        feed_dict = {}
        
        rgb_saver.restore(sess, checkpoints['rgb_imagenet'])
        flow_saver.restore(sess, checkpoints['flow_imagenet'])
        
        tf.logging.info('RGB checkpoint restored')
        tf.logging.info('Flow checkpoint restored')
       
        rgb_arr = rgb_batch[np.newaxis, :]
        rgb_arr = (rgb_arr/255.0)*2-1
        flow_arr = flow_batch[np.newaxis, :]
        flow_arr = (flow_arr/255.0)*2-1

        feed_dict[rgb_input] = rgb_arr
        feed_dict[flow_input] = flow_arr

        rgb, flow = sess.run([rgb_feat, flow_feat], feed_dict=feed_dict)
    
    return rgb, flow


In [40]:
import cv2
cap = cv2.VideoCapture('data/v_CricketShot_g04_c01.avi')

if cap.isOpened() == False:
    print('Error opening video stream or file')

frames=[]

height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)

min_ = min(height, width)
ratio = float(256/float(min_))
new_w = int(ratio*width)
new_h = int(ratio*height)
i=0
while cap.isOpened():
    
    ret, frame = cap.read()
#     name = str(i)+'.jpg'
#     cv2.imwrite('data/frames/'+name, frame)
    if ret == True:
        # frames+=1
        i=i+1
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
        frame = (frame/255.0)*2-1
        frame = frame[int((new_h-224)/2):int((new_h+224)/2),int((new_w-224)/2):int((new_w+224)/2),:]
        frames.append(frame)
    else:
        break
print(np.array(frames).shape)
cap.release()
cv2.destroyAllWindows()

(80, 224, 224, 3)


In [43]:
import numpy as np
a=np.load('data/v_CricketShot_g04_c01_rgb.npy')
frames = np.array(frames)
print(frames.shape)
# frames = frames[0]
print(frames.shape, a.shape)
print(np.linalg.norm(a[0,0].flatten()-frames[0].flatten()))

(80, 224, 224, 3)
(80, 224, 224, 3) (1, 79, 224, 224, 3)
66.80770606607665


In [26]:
a[0,0,:,0,0]

array([ 0.62228775,  0.22845614, -0.07612872, -0.0310626 , -0.02398157,
        0.14633   ,  0.5618305 ,  0.6487572 ,  0.125471  , -0.2573926 ,
        0.03273392,  0.45883274,  0.6471859 ,  0.09479284, -0.08804142,
       -0.00917238, -0.00297922, -0.01075757, -0.03898668, -0.04991221,
       -0.05772483, -0.06553745, -0.0666666 , -0.06001365, -0.0654459 ,
       -0.05780035, -0.06267869, -0.08355308, -0.10699093, -0.10231721,
       -0.10269183, -0.11378253, -0.10339981, -0.10047007, -0.10047007,
       -0.10675681, -0.1083132 , -0.1083132 , -0.11450839, -0.11615634,
       -0.11002219, -0.1083132 , -0.11438632, -0.11615634, -0.11615634,
       -0.1221379 , -0.12399948, -0.12399948, -0.11810946, -0.11817801,
       -0.12268025, -0.1317985 , -0.1403166 , -0.14437139, -0.15080482,
       -0.15294111, -0.15294111, -0.15294111, -0.15294111, -0.15294111,
       -0.15294111, -0.15294111, -0.15294111, -0.15294111, -0.15294111,
       -0.15294111, -0.15294111, -0.15294111, -0.16350031, -0.16

In [1]:
# Compare PIL and cv2
import numpy as np
import cv2
from PIL import Image

img_cv2BGR=cv2.imread('/home/june/mnt2/th14_feature_i3d/th14_raw/val_optical_flow_rgb/video_validation_0000684/img_00513.jpg')
img_cv2RGB = cv2.cvtColor(img_cv2BGR, cv2.COLOR_BGR2RGB)

img_pil = Image.open('/home/june/mnt2/th14_feature_i3d/th14_raw/val_optical_flow_rgb/video_validation_0000684/img_00513.jpg').convert('RGB')

arr_cv2 = np.array(img_cv2RGB)
arr_pil = np.array(img_pil)

print(sum(arr_cv2.flatten()-arr_pil.flatten()))

0


In [32]:
import cv2
def reshape_cv2(img, type):
    width, height = img.shape[0:2]
    min_ = min(height, width)
    ratio = float(256/float(min_))
    new_w = int(ratio*width)
    new_h = int(ratio*height)
    if type=='rgb':
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (new_w,new_h), interpolation=cv2.INTER_LINEAR)
    frame = (frame/255.0)*2-1
    frame = frame[(new_h-224)/2:(new_h+224)/2,(new_w-224)/2:(new_w+224)/2]
    
    return frame

a=cv2.imread('/home/june/mnt2/th14_feature_i3d/code/data/frames/0.jpg')
b=reshape_cv2(a, type='rgb')

(224, 198, 3)

In [30]:
import numpy as np
x=np.load('/home/june/code/kinetics-i3d/data/v_CricketShot_g04_c01_rgb.npy')
x=x[0,0]

In [31]:
print(sum(x.flatten()-b.flatten()) )

ValueError: operands could not be broadcast together with shapes (150528,) (133056,) 